In [54]:
import matplotlib
import pandas as pd
import numpy as np
from datetime import datetime
import theano
import theano.tensor as T
import crbm as C
import time

In [55]:
allData = pd.read_csv('household_power_consumption.txt',';',index_col=0,na_values='?',header=0,parse_dates=[[0, 1]],infer_datetime_format=True)

allData.dtypes

Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtype: object

In [56]:
allData

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_Time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0,1,17
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0,1,16
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0,2,17
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0,1,17
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0,1,17
2006-12-16 17:29:00,3.520,0.522,235.02,15.0,0,2,17
2006-12-16 17:30:00,3.702,0.520,235.09,15.8,0,1,17
2006-12-16 17:31:00,3.700,0.520,235.22,15.8,0,1,17
2006-12-16 17:32:00,3.668,0.510,233.99,15.8,0,1,17


In [57]:
#reduce the number of data coputing the max of each hour
groupedByH = allData.groupby(pd.TimeGrouper('H')).max()
groupedByH

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_Time,,,,,,,
2006-12-16 17:00:00,7.706,0.528,237.14,33.2,0,2,18
2006-12-16 18:00:00,6.752,0.202,238.28,29.0,0,37,18
2006-12-16 19:00:00,4.070,0.192,235.12,17.4,0,5,17
2006-12-16 20:00:00,3.496,0.170,237.54,14.8,0,0,18
2006-12-16 21:00:00,3.502,0.194,239.69,14.8,0,2,18
2006-12-16 22:00:00,3.464,0.200,242.04,14.6,0,2,18
2006-12-16 23:00:00,4.122,0.204,243.73,17.6,0,2,0
2006-12-17 00:00:00,3.038,0.334,244.03,12.6,0,2,0
2006-12-17 01:00:00,5.282,0.302,243.88,22.4,0,37,0


In [58]:
#split the dataset in traininig,validation and test set
def splitDataset(dt):
    y = dt.year
    if(y>=2006 and y<=2008):
        return 'training'
    if(y==2009):
        return 'validation'
    if(y==2010):
        return 'test'

splittedDataset = groupedByH.groupby(splitDataset)

trainingSet = splittedDataset.get_group('training')
validationSet = splittedDataset.get_group('training')
testSet = splittedDataset.get_group('training')

In [76]:
#number of visible unit
#n_dim = 1;
#numbero of hidden unit
n_hidden = 3;

#number of previous value to consider
delay = 3;

#learning rate
learning_rate = 0.001;

#the size of the trainining epoch
batch_size = 24;

#number of training epochs
training_epochs=10;

#size of training set
seqlen = 2400+delay

#training set
batchdata = trainingSet.Global_active_power[:seqlen].values

In [77]:
batchdata = np.asmatrix(batchdata).T

In [78]:
batchdata.shape

(2403, 1)

In [79]:
batchdata = theano.shared(np.asarray(batchdata, dtype=theano.config.floatX))

In [80]:
# compute number of minibatches for training, validation and testing
n_train_batches = batchdata.get_value(borrow=True).shape[0] / batch_size
n_dim = batchdata.get_value(borrow=True).shape[1]

In [81]:
# allocate symbolic variables for the data
index = T.lvector()    # index to a [mini]batch
index_hist = T.lvector()  # index to history
x = T.matrix('x')  # the data
x_history = T.matrix('x_history')

#theano.config.compute_test_value='warn'
#x.tag.test_value = np.random.randn(batch_size, n_dim)
#x_history.tag.test_value = np.random.randn(batch_size, n_dim*delay)

# initialize storage for the persistent chain
# (state = hidden layer of chain)

# construct the CRBM class
crbm = C.CRBM(input=x, input_history=x_history, n_visible=n_dim, n_hidden=n_hidden, delay=delay)

# get the cost and the gradient corresponding to one step of CD-15
cost, updates = crbm.get_cost_updates(lr=learning_rate, k=1)

In [82]:
batchdataindex = []
last = 0
batchdataindex += range(last + delay, last + seqlen)

permindex = np.array(batchdataindex)

In [83]:
train_crbm = theano.function([index, index_hist], cost,
           updates=updates,
           givens={
                    x: batchdata[index],
                    x_history: batchdata[index_hist].reshape((batch_size, delay * n_dim))
                  },
           name='train_crbm')

plotting_time = 0.
start_time = time.clock()

# go through training epochs
for epoch in xrange(training_epochs):

    # go through the training set
    mean_cost = []
    for batch_index in xrange(n_train_batches):
        #print '\n'
        # indexing is slightly complicated
        # build a linear index to the starting frames for this batch
        # (i.e. time t) gives a batch_size length array for data
        data_idx = permindex[batch_index * batch_size:(batch_index + 1)* batch_size]
        #print data_idx.shape
        # now build a linear index to the frames at each delay tap
        # (i.e. time t-1 to t-delay)
        # gives a batch_size x delay array of indices for history
        hist_idx = np.array([data_idx - n for n in xrange(1, delay + 1)]).T
        #print hist_idx.shape
        this_cost = train_crbm(data_idx, hist_idx.ravel())
        #print batch_index, this_cost
        mean_cost += [this_cost]

    print 'Training epoch %d, cost is ' % epoch, np.mean(mean_cost)

end_time = time.clock()

pretraining_time = (end_time - start_time)

print ('Training took %f minutes' % (pretraining_time / 60.))

Training epoch 0, cost is  9.30669277396
Training epoch 1, cost is  6.78324790639
Training epoch 2, cost is  4.95397460435
Training epoch 3, cost is  3.91289500914
Training epoch 4, cost is  3.45176619946
Training epoch 5, cost is  3.22989633307
Training epoch 6, cost is  3.09555556207
Training epoch 7, cost is  3.04849072697
Training epoch 8, cost is  3.00602508503
Training epoch 9, cost is  2.94447328712
Training took 0.003171 minutes
